# OpenAdvisor - Part 3 - Integration & Encoding

In the final section, we'll take all the features we gathered from previous sections and integrate them into a cohesive, serialized representation of the degree requirements. 

Beginning in part 3A, we'll narrow down the list of unknown requirements considerably by addressing those that contain hyperlinks. These hyperlinked requirements are usually general education requirements or some other special group of courses. Typically the hyperlinks reference a table containing a list of course options for that requirement. We'll scrape these tables and apply the same processing we used in 2B to infer the table structure. In part 3B, we'll combine all these elements into a cohesive, standardized relational database schema that explicitly defines a degree's requirements. We'll then transform these requirements into a human-readable, serialized form.

### 3A. Special Groups and Gen. Ed. Requirements

Let's start by importing dependencies and data then defining regex patterns.

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup as bs
from thefuzz import process
from thefuzz import fuzz
import re
import unicodedata
from verticalprinter import v
from jupyter_functions import display_styled_table

tablerowhtml_re = re.compile(r'<tr.+?</tr>', flags=re.DOTALL)
tablerowclass_pattern = '(?:class=")([^ "]*)'
'(?<=class=")(.+?)(?=")'

degreedf = pd.read_pickle('degreesorganized.pkl')

Next, we'll extract all links found within the tables (that don't just open a bubble window) and standardize fragments so they include their directories too.

In [2]:
baseurl = degreedf.link[0][:degreedf.link[0].index('.edu')+4]
degreedf['links'] = degreedf.html.str.findall('(?<=<a href=")[^"]+(?=")')
# Modify fragments that are on the degree page so they include the entire directory
startswithhash = degreedf.links.apply(lambda x: sum([bool(re.match('#', string)) for string in x]) != 0).fillna(False)
degreedf['pageurl'] = degreedf.link.apply(lambda x: x[x.index('.edu')+4:])
degreedf.loc[startswithhash, 'links'] = degreedf.apply(lambda x: [x.pageurl + '/' + string for string in x.links],
                                                       axis=1)

# List of all unique links
alllinks = list(set([x for sublist in degreedf.links for x in sublist]))
v(sorted(alllinks))    # Pretty print

    0
--  ------------------------------------------------------------------------------------------------------------------
 0  /general-catalog/all-university-core-curriculum/aucc/#Foundations-Perspectives
 1  /general-catalog/all-university-core-curriculum/aucc/#advanced-writing
 2  /general-catalog/all-university-core-curriculum/aucc/#arts-humanities
 3  /general-catalog/all-university-core-curriculum/aucc/#artsandhumanities
 4  /general-catalog/all-university-core-curriculum/aucc/#biolocial-physical-sciences
 5  /general-catalog/all-university-core-curriculum/aucc/#biological-physical-awareness
 6  /general-catalog/all-university-core-curriculum/aucc/#biological-physical-science
 7  /general-catalog/all-university-core-curriculum/aucc/#biological-physical-sciences
 8  /general-catalog/all-university-core-curriculum/aucc/#diversity-global
 9  /general-catalog/all-university-core-curriculum/aucc/#diversity-global-awareness
10  /general-catalog/all-university-core-curriculum/aucc/#fo

If you'll notice, several of these links are mispelled and broken (for instance, there are four different spellings of "social-behavioral-sciences"). These broken links are endemic on CourseLeaf, and we'll need to fix these to get decent results. We'll use fuzzy matching to sort this out. We look for valid links, then compare these to broken links by generating a score based on Levenshtein distance, then replacing the broken link if it's above a threshold score. The scraping process is very similar to part 1, so please refer to that Jupyter notebook for more insight into those steps.

In [3]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

siblings = []
linklist = []
oldlinklist = []
# Loop through each link, update it if broken, scrape page elements, and append relevant sibling elements
for link in alllinks:
    oldlink = link
    if '.edu' in link:
        url = link
    else:
        url = baseurl+link
    driver.get(url)
    driver.execute_script(
        "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    time.sleep(.3)
    sitehtml = driver.page_source
    soup = bs(sitehtml, features='lxml')
    # Get a list of all 'a' tags (links)
    taglist = [tag['name'] for tag in soup.select('a[name]')]
    if '#' in url:    # Hashtag indicates a fragment
        groupheaderid = url[url.rindex('#')+1:]
        linkdirectory = link[:link.rindex('#')+1]
        headerchild = soup.find(None, {'name': groupheaderid})
        # Fix broken links (links to right page but wrong fragment)
        if headerchild is None:      # Means link is broken
            # Find matches using fuzzy matching (not ideal but works in lieu of more advanced NLP)
            match = process.extract(groupheaderid, taglist, scorer=fuzz.token_set_ratio, limit=1)
            if match[0][1] >= 60:        # This can be tweaked but 60 seems to provide very good results
                fixedheaderid = match[0][0]
                link = linkdirectory + fixedheaderid
                headerchild = soup.find(None, {'name': fixedheaderid})
            else:
                raise Exception('Cant find a good match for the url fragment')
        
        # Figure out whether the group header is in the element or in one of it's next siblings
        # Todo: Reformat this so the code isn't repetitive
        groupheader = headerchild.parent
        if groupheader.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            groupheader = headerchild.parent.parent
            if groupheader.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                groupheader = headerchild.parent.parent.parent
                if groupheader.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                    groupheader = headerchild.nextSibling
                    if groupheader.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                        groupheader = headerchild.nextSibling.nextSibling
                        if groupheader.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                            raise Exception('Cant find header associated with fragment')
        siblings.append(str(groupheader))           # Groupheader is the first sibling element
        linklist.append(link)                       # Linklist contains the fixed links
        oldlinklist.append(oldlink)                 # Oldlinklist contains the original links
        headerlevel = int(groupheader.name[1])      # Headerlevel is used to determine the group of siblings
        for sibling in groupheader.next_siblings:
            if sibling.name is None:
                continue
            if sibling.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                if int(sibling.name[1]) <= headerlevel:     # If you reach a lower-level header, no more siblings exist
                    break
            elif sibling.name == 'div':                     # Drill down into div to get more sub-elements
                for child in sibling.children:
                    if child.name is None:
                        continue
                    if child.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                        if int(sibling.name[1]) <= headerlevel:
                            break
                    elif child.nam == 'div':
                        raise Exception('Increase the number of times you can drill down into divs')
                    siblings.append(str(child))
                    linklist.append(link)
                    oldlinklist.append(oldlink)
            else:
                siblings.append(str(sibling))
                linklist.append(link)
                oldlinklist.append(oldlink)
driver.close()

In the following steps, we take these HTML elements, extract the relevant data, append this data to the HTML tables, then save it all within a dataframe.

In [4]:
# Make a dataframe of these elements and links and organize/clean up
df = pd.DataFrame({'flink': linklist, 'oldlink': oldlinklist, 'html': siblings})        # 'flink' is fragment link
if not df.empty:
    # Get html class and extract all sc_courselists     Todo: clean up naming scheme like in script 3
    df['htmlclass'] = df.html.str.extract('(?<=<)(.+?)(?=( |>))')[0]
    is_table = df.htmlclass.eq('table')
    df.loc[is_table, 'htmlclass'] = df.html.str.extract('(?<=class=")(.+?)(?=")').iloc[:, 0].str.split().str[0]
    df = df[df.groupby('flink').htmlclass.transform(lambda x: (x == 'sc_courselist').any())]
    df = df[df.htmlclass.eq('sc_courselist')]

    tablesseries = df.html.apply(lambda x: pd.read_html(x)[0])      # Unpack tables into an exploded series
    tablesseries.rename('tables', inplace=True)
    tablesseries = tablesseries.apply(lambda x: x.fillna(''))
    tablesseries = tablesseries.apply(lambda x: x.applymap(lambda y: unicodedata.normalize('NFKC', str(y).strip(' \n')).
                                                           encode('ascii', 'ignore').decode('utf-8')))
    # Mark current rows as not being a table header, then move column index to row and mark those as table headers
    tablesseries = tablesseries.apply(lambda x: x.assign(headerflag=False))
    tablesseries = tablesseries.apply(lambda x: x.T.reset_index().T.reset_index(drop=True))
    tablesseries.apply(lambda x: x.insert(2, "coregroup", '') if len(x.columns) == 4 else None)
    tablesseries.apply(lambda x: x.set_axis(['code', 'title', 'coregroup', 'credits', 'headerflag'], axis=1,
                                            inplace=True))
    tablesseries = tablesseries.apply(lambda x: x.assign(headerflag=x.headerflag.ne(False), axis=1))
    tabledf = pd.concat([tablesseries, df], axis=1)

    # Assign ID's for each unique link (not oldlink) starting at 9000 (so they are distinct to requirement df id's)
    tabledf = tabledf.sort_values(by='flink')
    istoprow = pd.Series([False]*len(tabledf))
    istoprow.index = tabledf.index
    istoprow.loc[tabledf.flink.groupby(df.flink).head(1).index] = True
    tabledf['id'] = istoprow.cumsum() + 9000
    tabledf = tabledf.assign(id=list(range(len(df))))  # id is a unique number for each table

    # Broadcast tablewide info to tabledf
    tabledf.tables = tabledf.apply(lambda x: x.tables.assign(flink=x.flink), axis=1)
    tabledf.tables = tabledf.apply(lambda x: x.tables.assign(headertext=x.oldlink), axis=1)  # Todo: Fix this workaround
    tabledf.tables = tabledf.apply(lambda x: x.tables.assign(id=x.id), axis=1)
    # Convert entire table html into html of individual rows
    tabledf.tables = tabledf.apply(lambda x: x.tables.assign(
        html=[''] * (len(x.tables) - len(tablerowhtml_re.findall(x.html))) + tablerowhtml_re.findall(x.html)), axis=1)
    tabledf.tables = tabledf.apply(lambda x: x.tables.assign(rowclass=x.tables.html.str.extract(tablerowclass_pattern)),
                                   axis=1)
    # Convert series containing dataframes into one big dataframe and clean up
    geneddf = pd.concat(tabledf.tables.to_list())
    # Make id unique from degree df id's
    geneddf.id = geneddf.id + 9000

    # Fill in blank columns so it processes correctly in script 6
    geneddf['degree'] = 'GENEDS'
    geneddf['siblingheaders'] = ''
    geneddf['superscripts'] = ''
    geneddf['tabnumber'] = ''
    geneddf['pagenumber'] = ''
    geneddf['pagetitle'] = 'GENEDS'
    geneddf['link'] = ''
    geneddf['htmlclass'] = 'sc_courselist'
else:
    geneddf = df
    
geneddf_only_important_columns = geneddf[['code', 'title', 'credits', 'flink', 'id']]
display_styled_table(geneddf_only_important_columns.head(20))

,code,title,credits,flink,id
0,Code,Title,Credits,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
1,BUS 300,Business Writing and Communication (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
2,CHEM 301,Advanced Scientific Writing--Chemistry (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
3,CO 300,Writing Arguments (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
4,CO 301A,Writing in the Disciplines: Arts and Humanities (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
5,CO 301B,Writing in the Disciplines: Sciences (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
6,CO 301C,Writing in the Disciplines: Social Sciences (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
7,CO 301D,Writing in the Disciplines: Education (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
8,CO 302,Writing in Digital Environments (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000
9,JTC 300,Strategic Writing and Communication (GT-CO3),3,/general-catalog/all-university-core-curriculum/aucc/#Advanced-Writing,9000


Now, we will run this dataframe through the same organization process that we covered in 2B.

In [5]:
# Rename the primary degree requirement df so it's not overwritten
main_degreedf = pd.read_pickle('degreesorganized.pkl')
main_degreedf.to_pickle('main_degreesorganized.pkl')

# Save geneds as degreetable so it's processed in degree_organizer
geneddf.to_pickle('degreetables.pkl')

# Re-run degree organizer with geneds instead
import OA_6_Degree_Organizer

geneddf_organized = pd.read_pickle('degreesorganized.pkl')
df_only_important_columns = geneddf_organized[['code', 'credits', 'headercodes', 'rowtype', 'unknownreq', 'degree', 'link']]
display_styled_table(df_only_important_columns.head(20))

,code,credits,headercodes,rowtype,unknownreq,degree,link
0,Code,Credits,,table header,False,GENEDS,
1,_BUS300_,3,_3_credits_,elective,False,GENEDS,
2,_CHEM301_,3,_3_credits_,elective,False,GENEDS,
3,_CO300_,3,_3_credits_,elective,False,GENEDS,
4,_CO301A_,3,_3_credits_,elective,False,GENEDS,
5,_CO301B_,3,_3_credits_,elective,False,GENEDS,
6,_CO301C_,3,_3_credits_,elective,False,GENEDS,
7,_CO301D_,3,_3_credits_,elective,False,GENEDS,
8,_CO302_,3,_3_credits_,elective,False,GENEDS,
9,_JTC300_,3,_3_credits_,elective,False,GENEDS,


### 3B - Degree Requirement Integration

The final section accomplishes two main steps. First we will fill in as many of the unknown requirements as possible, including the gen eds we gathered in 3A, and also ranges of course codes, sub-groups of courses, and elective groups specific to a degree. Second, we do one final step of transformation, converting the tables and all their requirements into a serialized form. Below is a short example of a serialized table code we'll be creating and the table it represents:

<strong>\_1_credits__AB120_<1><2> | \_1_credits__AB130_<1><2> | \_3_credits__AREC202_ | \_4_credits__CHEM107_ | \_1_credits__CHEM108_ | \_3_credits__CO150_ | \_1_groups_ \_8_credits_{{\_LIFE102_ | \_LIFE103_} | {\_BZ110_ | \_BZ111_ | \_BZ120_}} | \_6_credits__table_9011_ | \_3_credits__0000_</strong>

![alt text](ag_bio_semester1.jpg "example table")

Headercodes (e.g., "\_1_credits_") apply either to individual requirements (e.g., "\_AB130_") or groups of requirements (indicated by brackets). Superscripts are referenced with angle brackets (eg. <1><2>), and corresponding superscript definitions are located in the
dataframe (however are not interpreted). "|" represents a generic separation representing "AND" (when seperating requirements with headercodes) or "OR" (when seperating requirements without individual headercodes; i.e., the bracketed groups above). The codes \_table_9011_ and \_0000_ reference another table's ID. These ID's will be formalized in this section. Requisites that are not parsed are reprinted as plain text within brackets (e.g., \_3_credits_{Lower-div. Written Communication}).


As usual, we'll begin by importing dependencies and working data, defining regex patterns, and doing some light housekeeping

In [7]:
from verticalprinter import v
import re
import os
from listtopattern import listtopattern
import json
import difflib
from random import sample

df = pd.read_pickle('main_degreesorganized.pkl')             # Output from first run of script 6
coursedf = pd.read_pickle('organizedcoursedescriptions.pkl')  # Output from Part 2A (Jupyter notebook)
geneddf = pd.read_pickle('degreesorganized.pkl')              # Output from script 7 followed by second run of script 6

with open('cnum_pattern.json') as infile:
    cnum_pattern = json.load(infile)                           # School-specific course code regex patterns
with open('cdept_pattern.json') as infile:
    cdept_pattern = json.load(infile)

or_pattern = r'(?: or | ?/ ?| ?\| ?)'
and_pattern = r'(?: and | ?& ?)'

# Concatenate geneddf and df into one dataframe
if not geneddf.empty:
    df = pd.concat([df, geneddf]).reset_index(drop=True)

# Move header superscripts so they aren't processed
superscriptcodes = df.code.str.extract('((?:<.>)+)').iloc[:, 0]
df.loc[superscriptcodes.notna() & df.headerlevel.notna(), 'headercodes'] = df.headercodes + superscriptcodes
df.loc[superscriptcodes.notna() & df.headerlevel.isna(), 'sscriptvalues'] = superscriptcodes
df.code = df.code.str.replace('<.>', '', regex=True)
df.codecopy = df.codecopy.str.replace('<.>', '', regex=True)

Remember that within one degree requirement page, we can have multiple different tables. There is typically a primary degree requirement table (courselist), a four year plan (plangrid), and sometimes tables that contain lists of electives (electivelists) that allow for more options within the courselist and plangrids. The electivelists are essentially ancillary tables to the main course requirement tables. The electivelists are usually referenced within the courselist (and occasionaly the plangrid), so we'll need to explicitly create a link between these tables. 

We'll do this by attempting to match the electivelist's title with the items in the courselist and plangrid. Typically the names aren't a perfect match, so we'll use fuzzy matching for this step.

In [8]:
# Link elective tables to their degree plans
# Determine the HTML h element that applies to each table (essentially its title)
df['tableheader'] = df.headertext.apply(lambda x: x[x.rindex(' : ') + 3:].strip() if ' : ' in x else x)
df.tableheader = df.tableheader.str.replace('<..?>|[(][^()]*[)]', '', regex=True)  # Remove superscripts and parentheses
df.tableheader = df.tableheader.str.replace(r'\b((select |choose )?\d\d? )?credits?\b', '',
                                            regex=True, flags=re.IGNORECASE)  # Remove credit requirements too
# Cleancode is the original plaintext without redundancies or other superlatives that may interfere with name matching
df['cleancode'] = df.codecopy.str.replace('<..?>|[(][^()]*[)]', '', regex=True)
not_names_pattern = r'\b((see )?(lists? |electives? |groups? |courses? |requirements? |listed |see )below)|((select |choose )?\d\d? )?credits?\b'
df.cleancode = df.cleancode.str.replace(not_names_pattern, '', regex=True, flags=re.IGNORECASE)
df['cleancodecopy'] = df.codecopy
df['matchscore'] = 0

# Elective tables are generally the tables that accompany degree requirements and indicate concentrations and options
electivetabledf = df[~df.containssum].groupby('id').agg({'degree': 'first', 'tableheader': 'first'})
electivetabledf = electivetabledf.reset_index(drop=True)

# Loop through each elective table and find requirements whose title matches the table's title
for index, degree, tableheader in electivetabledf.itertuples():
    codelist = df.loc[df.degree.eq(degree) & df.containssum & df.unknownreq, 'cleancode'].unique()
    if codelist.shape == (0,):
        continue
    matches = process.extract(tableheader, codelist, scorer=fuzz.token_sort_ratio, limit=10)
    for match in matches:
        if match[1] >= 70:                  # Todo: Fuzzy matching needs to be replaced with better NLP
            codematch = match[0]
            matchscore = int(match[1])      # Save match score so the match can be replaced if better match is found
            ismatchanddegree = df.matchscore.lt(matchscore) & df.degree.eq(degree)
            df.loc[df.cleancode.eq(codematch) & ismatchanddegree, 'tableheadermatch'] = tableheader
            df.loc[df.cleancode.eq(codematch) & ismatchanddegree, 'codematch'] = codematch
            df.loc[df.cleancode.eq(codematch) & ismatchanddegree, 'unknownreq'] = False
            df.loc[df.tableheader.eq(tableheader) & ismatchanddegree, 'ismatched'] = True
            # Replace df.code with tablecode
            df.loc[df.cleancode.eq(codematch) & ismatchanddegree, 'code'] = \
                '_table_' + df.loc[df.tableheader.eq(tableheader) & (df.degree.eq(degree)), 'id'].iloc[0].zfill(4) + '_'
            df.loc[df.cleancode.eq(codematch) & ismatchanddegree, 'matchscore'] = matchscore

Let's take a look at these replacements. Each table within this dataframe has a primary key (e.g., "\_table_0030_"), which is precisely what we replace the plain text entry with. For the printout below, "code" represents this primary key, "tableheadermatch" is the title of that key's table, and "codematch" is the plain text entry that was replaced.

In [9]:
# Print a sample of replacements  
df_replacements = df.loc[df.codematch.notna(), ['code', 'tableheadermatch', 'codematch']]
display_styled_table(df_replacements.loc[sample(df_replacements.index.to_list(), k=10)])

,code,tableheadermatch,codematch
14094,_table_0349_,Upper-Division Art History Courses,Upper-Division Art History
3951,_table_0120_,Interdisciplinary: International Business Group 2 – Global Focus,International Business Group 2 - Global Focus
3617,_table_0104_,Immersion: International Business Group 3 – Experiential Learning Requirement,International Business Group 3 - Experiential Learning Requirement
4134,_table_0125_,Immersion: International Business Group 3 – Experiential Learning Requirement,International Business Group 3 - Experiential Learning Requirement
19480,_table_0528_,Methods Support Option,Support Option
26902,_table_0728_,Engineering and Related Courses Elective List,Engineering and Related Courses Elective List Select a minimum of
1619,_table_0042_,Horticulture Electives,Horticulture Electives
3147,_table_0086_,Immersion: International Business Group 3 – Experiential Learning Requirement,International Business Group 3 - Experiential Learning Requirement
3969,_table_0121_,Immersion: International Business Group 3 – Experiential Learning Requirement,International Business Group 3 - Experiential Learning Requirement
19791,_table_0542_,Methods Support Option,Support Option


Now we'll make a new dataframe, gdf, which will eventually contain all the courses contained in gen ed requirement groups and all other special course groups (the tables we collected in part 3A) and course codes that represent ranges of courses (e.g., "MAT 3XX"). 

In [10]:
# Get coursegroups from course descriptions
coursedf.coursegroups = coursedf.coursegroups.apply(lambda x: ' | '.join(x))
allcoursegroupstring = ' | '.join(coursedf.coursegroups.to_list())

# Create new group dataframe, gdf to hold group names, their courses, and a unique ID
gdf = pd.DataFrame()
gdf = gdf.append({'group': 'electives', 'id': '_0000_', 'code': np.nan}, ignore_index=True)

We'll replace all the variations of the term "electives", then replace any code which solely contains one of those representations with the id for the electives group (which essentially represents every course in the catalog). Electives will have the id "\_0000_".

In [11]:
# Fix variations of electives
electivewords = [r'((department )?approved |selected |required |free |general )?electives?( or \w+\Z)?']
df.cleancode = df.cleancode.str.replace(listtopattern(electivewords), 'electives', regex=True, flags=re.IGNORECASE)
df.loc[df.cleancode.str.fullmatch('electives'), 'unknownreq'] = False
df.loc[df.cleancode.str.fullmatch('electives'), 'code'] = '_0000_'
df.loc[df.cleancode.str.fullmatch('electives'), 'cleancode'] = '_0000_'

We'll also need to deal with course codes that represent a range of courses (e.g., "MAT 3XX" or "ENG 1XX-4XX") and create an explicit list of courses that correspond to these groups. We'll do this by first standardizing the plain text representations of these ranges, make a list of all the unique ranges, then create a list of all the courses within those individual ranges, and finally replace the plain text ranges with a foreign ID that corresponds to the explicit list of courses.

In [12]:
# Standardize general number code requirements (e.g. MAT 3XX --> MAT _cnum_3xxx_)
df.cleancode = df.cleancode.str.replace(r'([*][*][*][*]?|XXXX?|xxxx?|____?)', r'_cnum_xxxx_', regex=True)
df.cleancode = df.cleancode.str.replace(r'(\d)(?:\*\*\*?|XXX?|xxx?|___?)', r'_cnum_\1xxx_', regex=True)
# Standardize general department code requirements (e.g. MAT _cnum_3xxx_ --> _dept_MAT_ _cnum_3xxx_)
departmentlist = coursedf.dept.unique().tolist()
df.cleancode = df.cleancode.str.replace(listtopattern(departmentlist), r'_dept_\1_', regex=True)
# Combine general depts and nums (e.g. _dept_MAT_ _cnum_3xxx_ --> _MAT_3xxx_)
df.cleancode = df.cleancode.str.replace(r'\b_dept_([A-Z]+)_ ?-? ?_cnum_([x\d]+)_\b', r'_\1_\2_', regex=True)
gencode_pattern = r'\b_[A-Z]+_[x\d]+_\b'
# Standardize slash and 'or' separated course ranges (e.g. _MAT_3xxx_ or _cnum_4xxx --> _MAT_3xxx_ | _MAT_4xxx_)
# Todo: Fix so it replaces for groups greater than 2 (maybe do this in implicit to explicit section in script 6)
df.cleancode = df.cleancode.str.replace(r'\b(_[A-Z]+_)([x\d]+_)\b' + or_pattern + r'_cnum_([x\d]+_)', r'\1\2 | \1\3',
                                        regex=True)
df.cleancode = df.cleancode.str.replace(r'\b(_[A-Z]+_)([x\d]+_)\b' + ' ?- ?' + r'_cnum_([x\d]+_)', r'\1\2 - \1\3',
                                        regex=True)

# Make a list of all the course range codes (i.e gencodes)
gencode_list = list(set([x for sublist in df.cleancode.str.findall(gencode_pattern).to_list() for x in sublist]))
gencodes = pd.DataFrame({'group': gencode_list})
if not gencodes.empty:
    # Extract the departments and course numbers for gencodes
    gencodes['cdept'] = gencodes.group.apply(lambda x: x[1:x[1:].index('_') + 1])
    gencodes['cnum'] = (gencodes.group.apply(lambda x: x[x[1:].index('_') + 2:-1]) + '?').str.replace('x', r'\d',
                                                                                                      regex=True)
    # Extract the lists of courses for each gencode from coursedf (e.g. MAT3XX --> MAT301, MAT302, MAT311, etc.)
    gencodes['code'] = gencodes.apply(lambda x: (x.cdept + coursedf.loc[coursedf.dept.eq(x.cdept) & coursedf.number.str.fullmatch(x.cnum), 'number']).to_list(), axis=1)
    # Add these course groups to gdf as a '|' separated string
    gendf = pd.DataFrame([gencodes.group, gencodes.code.apply(lambda x: ' | '.join(x))]).T
    gdf = pd.concat([gdf, gendf]).reset_index(drop=True)
    gdf.id = gdf.index.to_series().apply(lambda x: '_' + str(x).zfill(4) + '_')     # Give each group a unique ID number

    # Replace all references to gencodes with their respective ID number (i.e. group code)
    for code in gencodes.group.to_list():
        df.loc[df.cleancode.str.fullmatch(code), 'unknownreq'] = False
        df.loc[df.cleancode.str.fullmatch(code), 'code'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]
        df.loc[df.cleancode.str.fullmatch(code), 'cleancode'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]

# Make more gencodes for combinations of gencodes (eg. 'MAT3XX or MAT4XX')
isgencodeor = df.cleancode.str.fullmatch(gencode_pattern + '(' + or_pattern + gencode_pattern + ')+')
if isgencodeor.any():
    gencodeors = pd.DataFrame({'group': df.loc[isgencodeor, 'cleancode'].unique().tolist()})
    gencodeors.group = gencodeors.group.str.replace(or_pattern, ' | ', regex=True)
    gencodeors['grouplist'] = gencodeors.group.apply(lambda x: x.split(' | '))
    gencodeors['code'] = \
        gencodeors.apply(lambda x: gdf.loc[gdf.group.isin(x.grouplist)].agg(lambda y: ' | '.join(y)).code, axis=1)
    gencodeors.drop(columns='grouplist', inplace=True)
    gdf = pd.concat([gdf, gencodeors]).reset_index(drop=True)
    gdf.id = gdf.index.to_series().apply(lambda x: '_' + str(x).zfill(4) + '_')
    # Replace references to gencode combos in df.cleancode with their ID's
    for code in gencodeors.group.to_list():
        df.loc[df.cleancode.str.fullmatch(code), 'unknownreq'] = False
        df.loc[df.cleancode.str.fullmatch(code), 'code'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]
        df.loc[df.cleancode.str.fullmatch(code), 'cleancode'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]

# Make gencodes for ranges of gencodes (e.g. _MAT_2xxx_ - _MAT_4xxx_)
isgencoderange = df.cleancode.str.fullmatch(gencode_pattern + ' ?- ?' + gencode_pattern)
if isgencoderange.any():
    gencoderangers = pd.DataFrame({'group': df.loc[isgencoderange, 'cleancode'].unique().tolist()})
    gencoderangers['firstcode'] = gencoderangers.group.apply(lambda x: x.split(' - ')).apply(lambda x: x[0])
    gencoderangers['secondcode'] = gencoderangers.group.apply(lambda x: x.split(' - ')).apply(lambda x: x[1])
    # Determine which digit varies between firstcode and secondcode
    # Todo: Generalize this so it works on more than one digit variation
    gencoderangers['ndiff'] = gencoderangers.apply(lambda x: ''.join(list(difflib.ndiff(x.firstcode, x.secondcode))),
                                                   axis=1)
    gencoderangers['ndiff'] = gencoderangers.ndiff.str.replace('  ', '')
    gencoderangers['cnum'] = gencoderangers.ndiff.str.replace(r'_[A-Z]+_- (\d)\+ (\d)xxx_', r'[\1-\2]\\d\\d\\d?',
                                                              regex=True)
    gencoderangers['cdept'] = gencoderangers.firstcode.str.extract(r'_([A-Z]+)_')
    gencoderangers['code'] = gencoderangers.apply(lambda x: (x.cdept + coursedf.loc[coursedf.dept.eq(x.cdept) & coursedf.number.str.fullmatch(x.cnum), 'number']).to_list(), axis=1)
    # Add the list of all courses within gencode range to gdf
    rangersdf = pd.DataFrame([gencoderangers.group, gencoderangers.code.apply(lambda x: ' | '.join(x))]).T
    gdf = pd.concat([gdf, rangersdf]).reset_index(drop=True)
    gdf.id = gdf.index.to_series().apply(lambda x: '_' + str(x).zfill(4) + '_')
    # Replace reference to gencode ranges in df.cleancode with their ID
    for code in gencoderangers.group.to_list():
        df.loc[df.cleancode.str.fullmatch(code), 'unknownreq'] = False
        df.loc[df.cleancode.str.fullmatch(code), 'code'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]
        df.loc[df.cleancode.str.fullmatch(code), 'cleancode'] = gdf.loc[gdf.group.eq(code), 'id'].iloc[0]


Let's view some of the entries of gdf. "group" is the plaintext that was replaced, "id" is the foreign key that it was replaced with, and "code" is the explicit list of courses.

In [13]:
display_styled_table(gdf.loc[sample(gdf.index.to_list(), k=10)])

,group,id,code
55,_BSPM_4xxx_,_0055_,BSPM415 | BSPM423 | BSPM424 | BSPM445 | BSPM450 | BSPM451 | BSPM462 | BSPM487 | BSPM492 | BSPM495 | BSPM496
68,_SOC_3xxx_ | _SOC_4xxx_,_0068_,SOC403 | SOC422 | SOC431 | SOC444 | SOC455 | SOC460 | SOC461 | SOC462 | SOC463 | SOC474 | SOC487 | SOC492 | SOC495 | SOC301 | SOC302 | SOC311 | SOC313 | SOC314 | SOC315 | SOC320 | SOC322 | SOC323 | SOC324 | SOC330 | SOC332 | SOC333 | SOC334 | SOC340 | SOC341 | SOC342 | SOC343 | SOC344 | SOC351 | SOC352 | SOC353 | SOC354 | SOC356 | SOC357 | SOC358 | SOC359 | SOC360 | SOC362 | SOC364 | SOC372 | SOC373 | SOC375
56,_PSY_3xxx_,_0056_,PSY300 | PSY305 | PSY310 | PSY315 | PSY316 | PSY317 | PSY320 | PSY325 | PSY327 | PSY328 | PSY330 | PSY335 | PSY340 | PSY341 | PSY345 | PSY350 | PSY352 | PSY354 | PSY360 | PSY362 | PSY364 | PSY366 | PSY370 | PSY371 | PSY384 | PSY392
6,_ECON_3xxx_,_0006_,ECON304 | ECON306 | ECON310 | ECON315 | ECON317 | ECON320 | ECON325 | ECON327 | ECON332 | ECON335 | ECON340 | ECON346 | ECON370 | ECON372 | ECON376 | ECON379
42,_SOC_xxxx_,_0042_,SOC100 | SOC105 | SOC205 | SOC210 | SOC220 | SOC253 | SOC270 | SOC271 | SOC275 | SOC301 | SOC302 | SOC311 | SOC313 | SOC314 | SOC315 | SOC320 | SOC322 | SOC323 | SOC324 | SOC330 | SOC332 | SOC333 | SOC334 | SOC340 | SOC341 | SOC342 | SOC343 | SOC344 | SOC351 | SOC352 | SOC353 | SOC354 | SOC356 | SOC357 | SOC358 | SOC359 | SOC360 | SOC362 | SOC364 | SOC372 | SOC373 | SOC375 | SOC403 | SOC422 | SOC431 | SOC444 | SOC455 | SOC460 | SOC461 | SOC462 | SOC463 | SOC474 | SOC487 | SOC492 | SOC495 | SOC500 | SOC501 | SOC502 | SOC503 | SOC510 | SOC511 | SOC540 | SOC555 | SOC562 | SOC564 | SOC566 | SOC610 | SOC612 | SOC613 | SOC614 | SOC630 | SOC631 | SOC633 | SOC639 | SOC660 | SOC661 | SOC662 | SOC663 | SOC664 | SOC665 | SOC666 | SOC667 | SOC668 | SOC669 | SOC671 | SOC695 | SOC696 | SOC699 | SOC752 | SOC784 | SOC787 | SOC795 | SOC799
66,_POLS_3xxx_ | _POLS_4xxx_,_0066_,POLS302 | POLS303 | POLS304 | POLS305 | POLS306 | POLS308 | POLS309 | POLS320 | POLS321 | POLS331 | POLS332 | POLS341 | POLS345 | POLS347 | POLS351 | POLS361 | POLS362 | POLS364 | POLS392 | POLS405 | POLS409 | POLS410 | POLS413 | POLS420 | POLS421 | POLS422 | POLS423 | POLS431 | POLS433 | POLS435 | POLS436 | POLS437 | POLS440 | POLS442 | POLS443 | POLS444 | POLS445 | POLS446 | POLS447 | POLS448 | POLS449 | POLS451 | POLS459 | POLS460 | POLS462 | POLS463 | POLS465 | POLS487 | POLS492 | POLS495 | POLS496
40,_ECON_xxxx_,_0040_,ECON101 | ECON202 | ECON204 | ECON211 | ECON212 | ECON235 | ECON240 | ECON304 | ECON306 | ECON310 | ECON315 | ECON317 | ECON320 | ECON325 | ECON327 | ECON332 | ECON335 | ECON340 | ECON346 | ECON370 | ECON372 | ECON376 | ECON379 | ECON404 | ECON410 | ECON435 | ECON440 | ECON442 | ECON444 | ECON460 | ECON463 | ECON474 | ECON484 | ECON487 | ECON492 | ECON495 | ECON501 | ECON504 | ECON505 | ECON506 | ECON510 | ECON515 | ECON520 | ECON530 | ECON535 | ECON540 | ECON541 | ECON563 | ECON570 | ECON604 | ECON606 | ECON635 | ECON640 | ECON663 | ECON695 | ECON698 | ECON699 | ECON704 | ECON705 | ECON706 | ECON715 | ECON720 | ECON735 | ECON740 | ECON741 | ECON742 | ECON760 | ECON770 | ECON771 | ECON772 | ECON784 | ECON793 | ECON795 | ECON799
1,_HIST_xxxx_,_0001_,HIST100 | HIST101 | HIST115 | HIST116 | HIST120 | HIST121 | HIST150 | HIST151 | HIST170 | HIST171 | HIST201 | HIST250 | HIST252 | HIST255 | HIST300 | HIST301 | HIST302 | HIST303 | HIST304 | HIST305 | HIST308 | HIST309 | HIST310 | HIST311 | HIST312 | HIST315 | HIST317 | HIST318 | HIST319 | HIST320 | HIST321 | HIST322 | HIST323 | HIST324 | HIST325 | HIST328 | HIST329 | HIST330 | HIST331 | HIST332 | HIST333 | HIST334 | HIST335 | HIST336 | HIST338 | HIST339 | HIST340 | HIST341 | HIST344 | HIST345 | HIST347 | HIST348 | HIST349 | HIST350 | HIST351 | HIST352 | HIST353 | HIST354 | HIST355 | HIST356 | HIST357 | HIST358 | HIST359 | HIST360 | HIST361 | HIST362 | HIST363 | HIST364 | HIST365 | HIST366 | HIST367 | HIST369 | HIST370 | HIST371 | HIST379 | HIST392 | HIST410 | HIST411 | HIST412 | HIST41

We'll now integrate the course groups we collected in section 3A (gen ed requirements and other special groups). We'll simply use the hyperlinks we used in 3A to link those back to the corresponding course listing tables. These tables have ID's (i.e. primary keys) that start at 9000 (e.g., _table_9003_).

In [14]:
# Replace references to gen ed requirements (these are the tables extracted in script 7)
geneddf2 = df.loc[df.id.apply(float) > 8999].copy()
geneddf2['oldlink'] = geneddf2.headertext         # Todo: Change this workaround so links arent in headertext
# Extract fragment links (used to id which table they belong to)
df['fragmentlink'] = df.html.str.extract('(?<=<a href=")([^"]+)(?=")', expand=False).fillna('')
# Replace the incorrect broken fragment links with their correced version (these were fixed in script 7)
startswithhash = df.fragmentlink.str.match('#')
df['pageurl'] = df.link.apply(lambda x: x[x.index('.edu')+4:] if '.edu' in x else '')
df.loc[startswithhash, 'fragmentlink'] = df.apply(lambda x: x.pageurl + x.fragmentlink, axis=1)
# Replace references to gen ed requirements with their their corresponding table ID
oldlinklist = geneddf2.oldlink.tolist()
idlist = geneddf2.id.tolist()
linkdict = dict(zip(oldlinklist, idlist))
df.loc[df.unknownreq & df.fragmentlink.notna(), 'cleancode'] = \
    df.fragmentlink.apply(lambda x: '_table_' + str(linkdict[x]) + '_' if x in linkdict else 'UNKNOWN')
df.loc[df.unknownreq & df.fragmentlink.notna(), 'code'] = \
    df.fragmentlink.apply(lambda x: '_table_' + str(linkdict[x]) + '_' if x in linkdict else 'UNKNOWN')
df.loc[df.unknownreq & df.fragmentlink.notna() & df.code.ne('UNKNOWN'), 'unknownreq'] = False

Occasionally the catalog contains ambiguous or invalid headers (such as below, where course codes are ambiguously a header to other courses). We'll flag these cases with the 'headererror' flag, and remove their header status.

In [15]:
# Fix misclassified headers (headers that now have codes in them)
coursegroup_pattern = r'(_\d\d\d\d_|_table_\d\d\d\d_|_' + cdept_pattern + cnum_pattern + '_)'
onlycodes_pattern = '{?' + coursegroup_pattern + '(( & | [|] |[{}])' + coursegroup_pattern + ')*}?(<..?>)*'
onlycodes = df.code.str.fullmatch(onlycodes_pattern)
notheaders = onlycodes & df.headerlevel.notna() & ~df.rowtype.eq('row header')

# Print notheaders
display_styled_table(df.loc[notheaders, ['code', 'title', 'link']])

df.degreeflags = df.degreeflags + notheaders.groupby(df.id).transform(lambda x: 'headererror ' if x.any() else '')
df.loc[notheaders, 'headerlevel'] = np.nan
# Replace tablecodes and groupcodes in lines that are in fact headers
df.loc[onlycodes & df.headerlevel.notna() & df.rowtype.eq('row header'), 'code'] = df.codecopy.str.replace(r'<..?>', '',
                                                                                                           regex=True)

,code,title,link
11059,_HDFS439_,Administration of Early Childhood Programs,https://catalog.colostate.edu/general-catalog/colleges/health-human-sciences/human-development-family-studies/human-development-family-studies-major-human-development-family-studies-concentration
11146,_HDFS439_,Administration of Early Childhood Programs,https://catalog.colostate.edu/general-catalog/colleges/health-human-sciences/human-development-family-studies/human-development-family-studies-major-leadership-entrepreneurial-professions-concentration
11250,_HDFS475_,Entrepreneurs and Leaders in Human Services,https://catalog.colostate.edu/general-catalog/colleges/health-human-sciences/human-development-family-studies/human-development-family-studies-major-pre-health-professions-concentration
11340,_HDFS475_,Entrepreneurs and Leaders in Human Services,https://catalog.colostate.edu/general-catalog/colleges/health-human-sciences/human-development-family-studies/human-development-family-studies-major-prevention-intervention-sciences-concentration


At this point, we've replaced plain text entries with primary and foreign keys for gen ed requirements, course code ranges, general electives, and ancillary elective tables. In 2B, we replaced plain text representations for all the requisites that contain course codes or combinations theorof. All of these replacements represent the vast majority of requirements contained in a college catalog, but we will still have some left over which were not parsed. Let's visualize some of these unclassified items. 

In [16]:
# Unknowns is used to visualize everything that doesn't get parsed
unknowns = df.codecopy[df.unknownreq]
print(unknowns.loc[sorted(sample(unknowns.index.to_list(), k=20))])

1230                      Department-approved Study Abroad
2206          Agricultural and Resource Economics Elective
4367                           Education Abroad experience
4474           _MATH141_ (or higher level calculus course)
5943                            Career Development Seminar
6655             ECE 5** - Any ECE course at the 500-level
7947                                       Regular Courses
9198                 Family and Consumer Sciences Elective
9276                                      Required Courses
10623                                           Statistics
14028                                         Art Elective
14030                          Upper-Division Art Elective
14191                          Upper-Division Art Elective
16296            HIST *** History, Upper-Division non U.S.
16413            HIST *** History, Upper-Division non U.S.
17258    Individualized Focus Area and/or Second Field ...
20768                                      Other Electiv

We'll convert these back to their (mostly) unmodified, original text since they remain unparsed and we want an easy to read plain text representation. We'll also calculate some basic stats on what was parsed and what wasn't.

In [17]:
# Revert unknown requirements back to original text and surround with brackets so reqcodes and superscripts make sense
df.loc[df.unknownreq, 'code'] = '{' + df.codecopy + '}'

# Calculate totals
totalreqs = sum(df.headerlevel.isna())
totalheaderreqs = sum(df.headerlevel.notna() & df.headercodes.ne(''))
# Calculate ratio of unknown requirements
totalunknowns = sum(df.unknownreq)
unknownratio = totalunknowns/totalreqs

print(str(totalreqs) + ' total degree requirements')
print(str(totalreqs - totalunknowns) + ' degree requirements parsed (' + str(round((1-unknownratio) * 100, 2)) + ')%')

df = df.reset_index(drop=True)

23890 total degree requirements
23127 degree requirements parsed (96.81)%


We can see that the unknowns represent less than 4% of the total (meaning we classified over 96% of the 22,000 requirements!). While we've parsed over 96% of entries, this doesn't mean that they were necessarily parsed correctly. We'll have to rely on manual verification for that. From manual checks (verifying a random sample of 100 entries at a time), I've found the CSU example to be over 99% accurate, and over 98% accurate for all the schools on average. I encourage you to choose one of the final encodings from the master list, click on it's corresponding link and check the accuracy if you're curious!

The one area that I have the least confidence in terms of accuracy is the fuzzy matching algorithm. We still should manually check the fuzzy match replacements to make sure they are accurate in the vast majority of cases, and adjust the target match score accordingly if not. We'll make a new dataframe to contain all of the replacements we did and visualize a sample (note that this table just shows the two matched strings, not the key that replaces the entry).

In [18]:
# Create a dataframe showing the codes that were replaced with tablecodes so they can be verified
df['groupname'] = df.code.apply(lambda x: gdf.group[gdf.id.eq(x)].iloc[0] if not gdf.group[gdf.id.eq(x)].empty
                                else np.nan)
df['tableid'] = '_table_' + df.id.apply(lambda x: x.zfill(4)) + '_'
df['tablename'] = df.apply(lambda x: df.headertext[df.tableid == x.code].iloc[0]
                           if not df.headertext[df.tableid == x.code].empty else np.nan, axis=1)
df.loc[df.code.str.fullmatch(r'_\d\d\d\d_'), 'codenames'] = df.groupname
df.loc[df.code.str.fullmatch(r'_table_\d\d\d\d_'), 'codenames'] = df.tablename

# This is used to visualize and double check all the group names that were matched via fuzzy matching
has_been_replaced = df.code.str.fullmatch(r'(_\d\d\d\d_|_table_\d\d\d\d_)')
replaced = df.loc[has_been_replaced][['codecopy', 'codenames']].groupby(df.codecopy, as_index=False).agg('first')

display_styled_table(replaced.loc[sorted(sample(replaced.index.to_list(), k=20))])

,codecopy,codenames
28,Computer Engineering Electives - Group 1 (see list below),Computer Engineering Electives 14-29 credits
29,Computer Engineering Electives - Group 2 and Group 3 (see list below),Computer Engineering Electives 14-29 credits
52,Early Childhood Professions Concentration Courses (See list below),Effective Fall 2021 : Early Childhood Professions Concentration Courses
61,Entomology Electives (see list below),Effective Fall 2020 : Entomology Electives
63,Foundation and Perspectives,/general-catalog/all-university-core-curriculum/aucc/#foundations-perspectives
71,"HIST *** History, upper-division U.S.","Effective Fall 2021 : History, Upper-Division U.S. Courses"
81,Horticulture Electives (see list below),Horticulture Electives
82,Horticulture Science Courses (see list below),Effective Fall 2020 : Horticultural Science Courses
94,MU ***,_MU_xxxx_
96,Math Electives (Select two courses from the Math Electives List below),Effective Fall 2021 : Math Electives List


Before the final step, we'll do some cleanup:

In [19]:
# Move superscripts back to df.code so they are linked
df.loc[df.headerlevel.isna() & df.sscriptvalues.notna(), 'code'] = df.code + df.sscriptvalues

# Convert ID column to table ID's
df.id = df.id.apply(lambda x: '_table_' + x.zfill(4) + '_')

The final step is really where the magic happens. We'll convert a degree requirement table into a serialized string that encapsulates all of the heirarchical details of the table and each corresponding entry. To accomplish this, we'll use the header level we calculated in 2B which determines the heirarchical structure of the table. We'll begin at the highest header level (headers of the least importance), collapse all the entries that fall under these headers and join them together. We'll then take the corresponding header codes (i.e. the requirements that apply to those collapsed entries) and append those to the collapsed requirements. We'll then move on to the next most important header level and repeat. This continues up the entire heirarchy until the whole table is represented as a single string, with brackets dilineating functional groups of requirements. We'll visualize some of these codes at the end of this notebook.

In [20]:
# Finally, serialize the code column
hlevels = df.headerlevel.dropna().unique()
hlevels.sort()
df = df.reset_index()
# Collapse groups one by one, starting with the ones with the highest header levels (most inner groups),
# then aggregate codes, surround with brackets, and attach requirements from header to front
for hlevel in hlevels[::-1]:
    # Froup everything by hlevel, then omit groups that don't include the current hlevel
    df['groups'] = ((df.headerlevel <= hlevel) | df.endofindent).cumsum()
    df.loc[df.headerlevel.groupby(df.groups).transform('first').ne(hlevel), 'groups'] = np.nan
    df.loc[df.groups.groupby(df.groups).transform('count') == 1, 'groups'] = np.nan
    firstismeta = df.rowtype.groupby(df.groups).transform('first') == 'metagroup header'
    # For metagroups, only use group headers
    df.loc[firstismeta & ~df.rowtype.isin(['group header', 'metagroup header']), 'groups'] = np.nan
    # Append headercodes to individual requirements
    df.loc[df.groups.notna() & df.headercodes.ne(''), 'code'] = df.headercodes + df.code
    # Aggregate all non header cells into their header cell, joining the codes with '|'
    aggcells = df.groupby('groups', as_index=False).agg(
        {'index': 'first', 'code': lambda x: '{' + ' | '.join(x[1:]) + '}' if len(x[1:]) != 1 else x[1:],
         'title': 'first', 'coregroup': 'first', 'credits': 'first', 'degree': 'first', 'link': 'first',
         'headertext': 'first', 'superscripts': 'first', 'htmlclass': 'first', 'id': 'first', 'html': 'first',
         'rowclass': 'first', 'codecopy': 'first', 'headercodes': 'first', 'containssum': 'first',
         'tableclass': 'first', 'degreeflags': 'first', 'degreetype': 'first', 'track': 'first',
         'maxdegreecredits': 'first', 'mindegreecredits': 'first', 'rowtype': 'first',
         'headerlevel': lambda x: 100, 'endofindent': 'first', 'groups': 'first'})
    df.drop(df[df.groups.notna()].index, inplace=True)
    df = pd.concat([df, aggcells]).sort_values('index').reset_index(drop=True)
    df.loc[df.headerlevel.shift(1).notna(), 'endofindent'] = False
    # Remove rows at the current hlevel without a code, headercode, or superscript (These are just descriptive names)
    df['keepers'] = (~df.headerlevel.eq(hlevel) | (
            df.code.str.contains(r' \| |<.>|' + cdept_pattern + cnum_pattern) | df.headercodes.ne('')))
    df = df.loc[df.headerlevel.isna() | df.keepers].reset_index(drop=True)


At this point we will append superscripts that apply to entire tables (they are in the table's title) to the serialized code.

In [21]:
# Append any table-wide superscripts to the entire code
code_plus_headersuperscripts = df.headertext.str.extract('((?:<.>)+)').iloc[:, 0] + '{' + df.code + '}'
df.loc[df.headertext.str.contains('<.>', regex=True), 'code'] = code_plus_headersuperscripts


That's it! We can now clean up and save the results. Let's also print out a sample of these codes and take a close look.

In [22]:
# Clean up and save
df = df[['tableclass', 'track', 'link', 'code', 'superscripts', 'degreeflags', 'degreetype', 'degree',
         'maxdegreecredits', 'mindegreecredits', 'id']]
with open('schoolname.json') as infile:
    schoolname = json.load(infile)
schooldirectory = 'Output_dataframes/' + schoolname
if os.path.isfile(schooldirectory + '/degreesserialized.pkl'):
    os.remove(schooldirectory + '/degreesserialized.pkl')
df.to_pickle(schooldirectory + '/degreesserialized.pkl')
if os.path.isfile(schooldirectory + '/groupsserialized.pkl'):
    os.remove(schooldirectory + '/groupsserialized.pkl')
gdf.to_pickle(schooldirectory + '/groupsserialized.pkl')
df.to_pickle('degreesserialized.pkl')
gdf.to_pickle('groupsserialized.pkl')

# # Push to PostgreSQL Amazon RDB
# with open("C:\config_files\settings.json") as infile:
#     sql_config = json.load(infile)
# connection = sqlalchemy.create_engine(sql_config)

# df.to_sql(schoolname + '_df', con=connection, index=False, if_exists='replace')
# gdf.to_sql(schoolname + '_gdf', con=connection, index=False, if_exists='replace')

# Print out 10 random degree requirements / elective tables
df_important_columns_only = df[['code', 'degree', 'id', 'tableclass', 'link']]
display_styled_table(df_important_columns_only.loc[sorted(sample(df.index.to_list(), k=10))])


,code,degree,id,tableclass,link
12,{_1_credits__AREC192_ | _3_credits__AREC202_ | _3_credits__CHEM103_ | _3_credits__CO150_ | _4_credits__CS110_ | _3_credits__ECON204_ | _1_credits__MATH117_ | _1_credits__MATH118_ | _1_credits__MATH124_ | _3-4_credits_ _1_courses_{_ANEQ101_ | _ANEQ102_ | _FTEC110_ | _HORT100_ | _SOCR100_} | _4_credits_{{_BZ110_ & _BZ111_} | _BZ120_ | _LIFE102_} | _3_credits__table_9019_ | {_3_credits__ACT205_ | _1_credits__AREC224_ | _3_credits__AREC305_ | _3_credits__MATH141_ | _3_credits__SPCM200_ | _3_credits__table_9000_ | _6_credits__table_9012_ | _3_credits__table_9024_ | _6_credits_{Agricultural Science Electives}<1>} | {_3_credits__AREC310_ | _3_credits_{_AREC335_ | _ECON335_} | _3_credits__ECON306_ | _3_credits__FIN305_ | _3_credits__MKT305_ | _3_credits__MKT362_ | _3_credits__STAT301_ | _3_credits_{_AREC311_ | _AREC325_ | {_AREC340_ | _ECON340_} | _AREC341_ | _AREC342_ | {_AREC346_ | _ECON346_} | _AREC375_ | _AREC415_ | _AREC442_ | {_AREC454_ | _REL454_} | _AREC461A_ | _AREC461B_} | _3_credits_{Agricultural Science Electives}<1> | _3_credits__0000_} | {_3_credits__AREC428_ | _6_credits_ _2_courses_{_AREC405_ | _AREC408_ | _AREC412_} | _3_credits_ _1_courses_{_AREC460_ | _AREC478_} | _6_credits_{_AREC311_ | _AREC325_ | {_AREC340_ | _ECON340_} | _AREC341_ | _AREC342_ | {_AREC346_ | _ECON346_} | _AREC375_ | _AREC405_ | _AREC408_ | _AREC412_ | _AREC415_ | _AREC442_ | {_AREC454_ | _REL454_} | _AREC460_ | _AREC461A_ | _AREC461B_ | _AREC478_} | _3_credits_{Agricultural Science Electives}<1> | _8-9_credits__0000_<2>}},Major in Agricultural Business,_table_0012_,plangrid,https://catalog.colostate.edu/general-catalog/colleges/agricultural-sciences/agricultural-resource-economics/business-major
117,{{_3_credits__ACT205_ | _3_credits_{_BUS205_ | _BUS260_} | _3_credits_ _1_courses_{_AREC202_ | _ECON202_ | _ECON204_}} | {_3_credits__FIN305_ | _3_credits__REL360_ | _6_credits_{_REL367_ | _REL430_ | {_REL454_ | _AREC454_} | _REL455_ | _REL460_}}},Minor in Real Estate,_table_0117_,courselist,https://catalog.colostate.edu/general-catalog/colleges/business/finance-real-estate/real-estate-minor
118,{_1_credits_{_BUS100_ | _BUS105_} | _1_credits__BUS201_<1> | _3_credits__CIS200_ | _3_credits__CO150_ | _3_credits__ECON202_ | _3_credits_{_MATH117_ | _MATH118_ | _MATH124_ | _MATH125_ | _MATH126_ | {_MATH141_ (or higher level calculus course)}} | _3_credits__table_9012_ | _4_credits__table_9015_ | _3_credits__table_9019_ | _9_credits__0000_ | {_3_credits__ACT210_ | _3_credits__ACT220_ | _3_credits__BUS220_<1> | _3_credits__BUS300_ | _3_credits__ECON204_ | _3_credits__STAT204_ | _3_credits__table_9015_ | _3_credits__table_9024_ | _6_credits__0000_} | {{All freshman and sophomore required courses must be completed prior to or concurrent with first enrollment in required junior and senior courses.} | _3_credits__BUS260_ | _3_credits__CIS370_ | _3_credits__MGT301_ | _3_credits__MGT310_ | _3_credits__MGT320_ | _3_credits__MGT350_ | _3_credits__MGT474_ | _9_credits__0000_} | {_3_credits__BUS479_ | _3_credits__FIN300_<2> | _3_credits__MGT374_ | _3_credits__MGT479_ | _3_credits__MKT300_<2> | _6_credits_ _2_courses_{_MGT410_ | _MGT411_ | _MGT473_ | _MGT476_ | _MGT486A_} | _6_credits__0000_<3>}},"Major in Business Administration, Human Resource Management Concentration",_table_0118_,plangrid,https://catalog.colostate.edu/general-catalog/colleges/business/management/business-administration-major-human-resource-management-concentration
157,{_1_credits__BIOM100_ | _4_credits__CHEM111_ | _1_credits__CHEM112_ | _3_credits__CO150_ | _4_credits__LIFE102_ | _4_credits__MATH160_ | _4_credits__MATH161_ | _3_credits__MECH103_ | _3_credits__MECH105_ | _5_credits__PH141_ | {_2_credits__BIOM200_ | _3_credits__CHEM113_ | _3_credits__CIVE260_ | _3_credits__CIVE261_ | _4_credits__MATH261_ | _4_credits__MATH340_ | _3_credits_ _1_groups_{_MECH200_ | {_MECH200A_ | _MECH200B_}} | _2_credits__MECH201_ | _3_credits__MECH231_ | _5_credits__PH142_} | {_4_cred

This process, from catalog html to a final serialized encoding, represents one of the biggest challenges in the OpenAdvisor project as a whole: collecting degree requirements and courses from diverse catalogs across a range of schools into one central, standardized database, which represents the vast majority of complex degree requirements. 

In future extensions of this project, I plan to use this database to cross reference between schools, finding all the schools that have courses that are a potential match for ones in degree requirements. This will eventually lead to the ability to query a list of all courses that can be replaced in a degree plan for cheaper alternatives at nearby or online schools. Much of the parsing functionality will likely be jettisoned (at least for now) to streamline the process of integrating as many schools as possible into a larger data set that can be used to train NLP models on. The current challenge is to develop an algorithm that can cross-reference courses between schools accurately. This will almost surely be a departure from fuzzy matching, possibly using word embeddings and clustering tailored to individual course codes. 

To be continued... 